In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import category_encoders as ce

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


from scipy.stats import uniform, randint
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score

# Import delle librerie essenziali
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


import os
import copy
import time
import random

from itertools import product



dataset_path = Path('EdgeIIoT-dataset.csv')
df = pd.read_csv(dataset_path)

: 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Supponiamo che X e y siano i tuoi dati sbilanciati
print(f"Distribuzione originale: {Counter(y)}")

# 1. Inizializzazione del campionatore
# sampling_strategy='auto' pareggia le classi 1:1
rus = RandomUnderSampler(random_state=42, replacement=False)

# 2. Applicazione dell'undersampling
X_resampled, y_resampled = rus.fit_resample(X, y)

print(f"Nuova distribuzione: {Counter(y_resampled)}")


ADABOOST

In [ ]:




classes, counts = np.unique(y_train, return_counts=True)
class_weights = {cls: 1.0/count for cls, count in zip(classes, counts)}
sample_weights = np.array([class_weights[label] for label in y_train])


# 3. Parametri AdaBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

ada_search = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid, cv=5, scoring='f1',n_jobs=-1)
ada_search.fit(X_train, y_train,sample_weight=sample_weights)



print("best accuracy",random_search_ada.best_score_)
print(ada_search.best_estimator_)


best_ada = ada_search.best_params_

ada = AdaBoostClassifier(**best_ada)
ada.fit(X_train, y_train, sample_weight=sample_weights)

# Predict continuous values
y_pred_ada = ada.predict(X_test)

#matrice di confusione
cm_ada = confusion_matrix(y_test, y_pred_ada)

# Use unique values for labels
labels = sorted(df['type'].unique())

plt.figure(figsize=(6, 4))
sns.heatmap(cm_ada, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels,
            yticklabels=labels)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matrice di Confusione - ada boost')
plt.show()


Random Forest

In [ ]:


# Definizione del modello e dei parametri da testare
rf = RandomForestClassifier(class_weight='balanced',random_state=42)
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Ricerca della migliore combinazione
grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy')
grid_rf.fit(X_train_balanced, y_train_balanced)

best_rf = grid_rf.best_estimator_
print(f"Migliori parametri RF: {grid_rf.best_params_}")

Rete Neurale

In [ ]:
# Configurazione
torch.manual_seed(42)
np.random.seed(42)
plt.style.use('default')

print("📦 Librerie caricate con successo!")
print(f"🔥 PyTorch: {torch.__version__}")

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"⚙️ Device: {device}")


print("📊 DIVISIONE DATASET:")
print(f"🏋️ Train:      {len(X_train):3d} campioni ({len(X_train)/len(X)*100:.1f}%)")
print(f"✅ Validation: {len(X_val):3d} campioni ({len(X_val)/len(X)*100:.1f}%)")
print(f"🎯 Test:       {len(X_test):3d} campioni ({len(X_test)/len(X)*100:.1f}%)")


# Verifica distribuzione classi
print("\n🔍 Distribuzione classi per set:")
for name, y_set in [("Train", y_train), ("Val", y_val), ("Test", y_test)]:
    dist = [np.sum(y_set == i) for i in range(2)]
    print(f"{name:5}: {dist} → {[d/len(y_set)*100 for d in dist]}")


# Conversione in tensori PyTorch
X_train_tensor = torch.FloatTensor(X_train)
X_val_tensor = torch.FloatTensor(X_val) 
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.LongTensor(np.array(y_train, dtype=np.float32))
y_val_tensor = torch.LongTensor(np.array(y_val, dtype=np.float32))  
y_test_tensor =  torch.LongTensor(np.array(y_test, dtype=np.float32))  

print(f"\n🔄 TENSORI PYTORCH:")
print(f"X_train: {X_train_tensor.shape}")
print(f"X_val:   {X_val_tensor.shape}")
print(f"X_test:  {X_test_tensor.shape}")




def fix_random(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

val_dataloader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=y_val_tensor.shape[0])
test_dataloader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=y_test_tensor.shape[0])

def get_model(input_size=83, dept=3, hidden_size=64, dropout_prob=0.2):

    # 1. Primo layer: Connette l'input al primo strato nascosto
    model = [nn.Linear(input_size, hidden_size), nn.ReLU() ]
    
    # 2. Layer intermedi (Hidden): Vengono aggiunti in un ciclo in base a 'dept'
    for i in range(dept):
        model.append(nn.Linear(hidden_size, hidden_size)) # Connessione lineare
        model.append(nn.ReLU())                           # Attivazione non lineare
        model.append(nn.Dropout(dropout_prob))            # Regolarizzazione (Dropout)
        
    # 3. Output layer: Connette l'ultimo layer nascosto alle 3 classi finali
    model.append(nn.Linear(hidden_size, 2))
    
    # nn.Sequential unisce la lista di layer in un unico modello ordinato
    return nn.Sequential(*model)


# --- GRID SEARCH SETUP ---
hidden_size = [128, 256]          
dropout_prob = [0.2, 0.3]         
dept = [3, 4]                     
batch_size = [16, 32]             
learning_rate = [0.001, 0.01]     

params = product(hidden_size, dropout_prob, dept, batch_size, learning_rate)

# Calcoliamo quante combinazioni dovremo testare
combinations = len(hidden_size)*len(dropout_prob)*len(dept)*len(batch_size)*len(learning_rate)
print("Numero totale di configurazioni da testare: ", combinations)


def train(model, train_dataloader, val_dataloader, device, hidden_size=3, dropout_prob=0.2, dept=2, epochs=100, batch_size=32, learning_rate=.001):
    """
    Esegue il training di un singolo modello.
    """
    # LOSS FUNCTION: Usiamo CrossEntropyLoss perché è un problema di Classificazione 
    criterion = nn.CrossEntropyLoss()
    
    # OPTIMIZER: Adam è l'algoritmo che aggiorna i pesi per minimizzare l'errore
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Liste per salvare lo storico degli errori
    train_loss = []
    val_loss = []

    # EARLY STOPPING:
    # ferma il modello se smette di migliorare sul validation set.
    best_model = None
    best_loss = np.inf   # Inizializziamo con infinito
    patience = 10        # Quante epoche aspettiamo se non migliora
    patience_counter = 0

    # --- CICLO DI TRAINING (EPOCHE) ---
    for epoch in range(epochs):
        epoch_start = time.time()
        epoch_loss = 0

        # Mettiamo il modello in modalità training (abilita Dropout, ecc.)
        model.train() 

        # Iteriamo sui batch di dati
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device) 
            
            optimizer.zero_grad()    # 1. Azzeriamo i gradienti precedenti
            y_pred = model(x)        # 2. Forward pass (previsione)
            loss = criterion(y_pred, y) # 3. Calcolo errore
            loss.backward()          # 4. Backward pass (calcolo gradienti)
            optimizer.step()         # 5. Aggiornamento pesi
            
            epoch_loss += loss.item()

        # Salviamo la loss media di training per questa epoca
        train_loss.append(epoch_loss / len(train_dataloader))

        # --- VALIDATION ---
        model.eval() # Modalità valutazione (disabilita Dropout)
        epoch_val_loss = 0
        with torch.no_grad(): # Disabilita calcolo gradienti (più veloce, meno memoria)
            for x, y in val_dataloader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                epoch_val_loss += loss.item()
        val_loss.append(epoch_val_loss / len(val_dataloader))

        # Stampa progressi ogni 10 epoche
        if (epoch+1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Train loss: {train_loss[-1]:.4f}, Val loss: {val_loss[-1]:.4f}, Time: {time.time()-epoch_start:.2f}s')

        # --- EARLY STOPPING CHECK ---
        # Se l'errore di validazione è il più basso visto finora, salviamo questo modello
        if val_loss[-1] < best_loss:
            best_loss = val_loss[-1]
            best_model = copy.deepcopy(model) # Creiamo una copia del modello attuale
            patience_counter = 0 # Resettiamo il contatore
        else:
            # Se non migliora, incrementiamo il contatore
            patience_counter += 1
            if patience_counter == patience:
                # Se abbiamo aspettato troppo interrompiamo.
                print("Early stopping...") 
                break

    print("Training terminato in {} epoche. Miglior validation loss: {}".format(epoch+1, best_loss))

    # Restituiamo il miglior modello trovato (non necessariamente l'ultimo)
    return best_model, train_loss, val_loss




def test_model(model, test_dataloader, device):
    """
    Valuta il modello finale sul Test Set per ottenere accuracy e predizioni.
    """
    model.eval() # Importante: modalità valutazione
    y_pred = []
    y_true = []
    
    with torch.no_grad():
        for x, y in test_dataloader:
            x , y = x.to(device), y.to(device)
            
            output = model(x)
            _, predicted = torch.max(output, 1) # Ottieni classe vincente
            
            y_pred.extend(predicted.cpu().tolist())
            y_true.extend(y.cpu().tolist())
            
    return y_pred, y_true





# --- CICLO PRINCIPALE GRID SEARCH ---
# Qui proviamo ad allenare una rete diversa per ogni combinazione di iperparametri
best_model = None
best_accuracy = 0
best_config = None
iter_count = 0 

# Iteriamo su tutte le combinazioni generate
for hidden_size, dropout_prob, dept, batch_size, learning_rate in params:
    iter_count += 1
    print(f'\n--- Iterazione {iter_count}/{combinations} ---')
    print(f'Configurazione: Hidden={hidden_size}, Drop={dropout_prob}, Dept={dept}, Batch={batch_size}, LR={learning_rate}')

    # 1. Creiamo un NUOVO modello con questa specifica configurazione
    model = get_model(X_train.shape[1], dept=dept, hidden_size=hidden_size, dropout_prob=dropout_prob)
    
    # Creiamo il DataLoader specifico (perché il batch_size cambia)
    train_dataloader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)

    # Dizionario utile per salvare la configurazione corrente
    config = {
        'hidden_size': hidden_size,
        'dropout_prob': dropout_prob,
        'dept': dept,
        'batch_size': batch_size,
        'learning_rate': learning_rate
    }

    # 2. Alleniamo il modello e otteniamo la versione migliore (grazie all'early stopping)
    trained_model, train_loss, val_loss = train(model, train_dataloader, val_dataloader, device, **config)

    # 3. Testiamo il modello allenato sul Test Set
    y_pred, y_true = test_model(trained_model, test_dataloader, device)
    test_acc = accuracy_score(y_true, y_pred)
    
    print(f'Test Accuracy: {test_acc:.4f} (Migliore attuale: {best_accuracy:.4f})')

    # 4. Confronto: È questo il modello migliore visto finora?
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_model = copy.deepcopy(trained_model) # Salviamo una copia
        best_config = config
        print("🏆 NUOVO RECORD!")


In [ ]:
print(f'Best config: {best_config}')
print(f'Best accuracy: {best_accuracy}')

y_pred, y_true = test_model(best_model, test_dataloader, device)
acc = accuracy_score(y_true, y_pred)
print(f'Final Accuracy score: {acc}')